In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline

model_name = "../../okt/model/gpt_code_v1_student"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained('gpt2')

/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
#数据读取

import time
import pandas as pd
import torch

dataset=pd.read_csv('testdata.csv')
print(dataset.head())

torch.cuda.empty_cache()

                                                Code  \
0  public int countClumps(int[] nums)\n{\n    int...   
1  public int countClumps(int[] nums)\n{\n    int...   
2  public int countClumps(int[] nums)\n{\n    int...   
3  public int countClumps(int[] nums)\n{\n    int...   
4  public int countClumps(int[] nums)\n{\n    int...   

                                              prompt  
0  Say that a "clump" in an array is a series of ...  
1  Say that a "clump" in an array is a series of ...  
2  Say that a "clump" in an array is a series of ...  
3  Say that a "clump" in an array is a series of ...  
4  Say that a "clump" in an array is a series of ...  


In [5]:
generate_codes=[]

for i in range(50):
    p=tokenizer(dataset.loc[i,'prompt'],return_tensors="pt")
    item=model.generate(**p,max_new_tokens=200)
    item=tokenizer.decode(item[0])
    generate_codes.append(item)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [6]:
gen=pd.DataFrame(generate_codes)
gen.to_csv('res_gpt2.csv')

In [7]:
print(len(generate_codes))
print(generate_codes)

50
['Say that a "clump" in an array is a series of 2 or more adjacent elements of the same value. Return the number of clumps in the given array.\npublic int countClumps(int[] nums)\r\n{\r\n    int clumps = 0;\r\n    for (int i = 0; i < nums.length - 1; i++)\r\n    {\r\n        if (nums[i] == nums[i+1])\r\n        {\r\n            clumps++;\r\n        }\r\n    }\r\n    return clumps;\r\n}\r\n<|endoftext|>', 'Say that a "clump" in an array is a series of 2 or more adjacent elements of the same value. Return the number of clumps in the given array.\npublic int countClumps(int[] nums)\r\n{\r\n    int clumps = 0;\r\n    for (int i = 0; i < nums.length - 1; i++)\r\n    {\r\n        if (nums[i] == nums[i+1])\r\n        {\r\n            clumps++;\r\n        }\r\n    }\r\n    return clumps;\r\n}\r\n<|endoftext|>', 'Say that a "clump" in an array is a series of 2 or more adjacent elements of the same value. Return the number of clumps in the given array.\npublic int countClumps(int[] nums)\r\n{

In [36]:
#用对应的真实代码进行对比

ground_truth_codes=[]
for i in range(50):
    ground_truth_codes.append(dataset.loc[i,'Code'])
print(len(ground_truth_codes))

50


In [19]:
#计算CodeBLUE

from codebleu import calc_codebleu

def compute_codebleu(ground_truth_codes, generated_codes):
    params='0.25,0.25,0.25,0.25'
    lang='java'
    codebleu_score= calc_codebleu(
                        references=ground_truth_codes, predictions=generated_codes, lang=lang)
    return codebleu_score

In [14]:
results = {}
codebleu_score= compute_codebleu(ground_truth_codes, generate_codes)
results['codebleu'] = codebleu_score

In [15]:
print(results['codebleu'])

{'codebleu': 0.10672738617872526, 'ngram_match_score': 0.06909512689426857, 'weighted_ngram_match_score': 0.11687508867745443, 'syntax_match_score': 0.09749670619235837, 'dataflow_match_score': 0.14344262295081966}


In [34]:
#计算Dist-N
import nltk

from collections import Iterable

import abc

def pad_sequence(
    sequence,
    n,
    pad_left=False,
    pad_right=False,
    left_pad_symbol=None,
    right_pad_symbol=None,
):
    """
    Returns a padded sequence of items before ngram extraction.

        >>> list(pad_sequence([1,2,3,4,5], 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
        ['<s>', 1, 2, 3, 4, 5, '</s>']
        >>> list(pad_sequence([1,2,3,4,5], 2, pad_left=True, left_pad_symbol='<s>'))
        ['<s>', 1, 2, 3, 4, 5]
        >>> list(pad_sequence([1,2,3,4,5], 2, pad_right=True, right_pad_symbol='</s>'))
        [1, 2, 3, 4, 5, '</s>']

    :param sequence: the source data to be padded
    :type sequence: sequence or iter
    :param n: the degree of the ngrams
    :type n: int
    :param pad_left: whether the ngrams should be left-padded
    :type pad_left: bool
    :param pad_right: whether the ngrams should be right-padded
    :type pad_right: bool
    :param left_pad_symbol: the symbol to use for left padding (default is None)
    :type left_pad_symbol: any
    :param right_pad_symbol: the symbol to use for right padding (default is None)
    :type right_pad_symbol: any
    :rtype: sequence or iter
    """
    sequence = iter(sequence)
    if pad_left:
        sequence = chain((left_pad_symbol,) * (n - 1), sequence)
    if pad_right:
        sequence = chain(sequence, (right_pad_symbol,) * (n - 1))
    return sequence



def ngrams(sequence, n, **kwargs):
    """
    Return the ngrams generated from a sequence of items, as an iterator.
    For example:

        >>> from nltk.util import ngrams
        >>> list(ngrams([1,2,3,4,5], 3))
        [(1, 2, 3), (2, 3, 4), (3, 4, 5)]

    Wrap with list for a list version of this function.  Set pad_left
    or pad_right to true in order to get additional ngrams:

        >>> list(ngrams([1,2,3,4,5], 2, pad_right=True))
        [(1, 2), (2, 3), (3, 4), (4, 5), (5, None)]
        >>> list(ngrams([1,2,3,4,5], 2, pad_right=True, right_pad_symbol='</s>'))
        [(1, 2), (2, 3), (3, 4), (4, 5), (5, '</s>')]
        >>> list(ngrams([1,2,3,4,5], 2, pad_left=True, left_pad_symbol='<s>'))
        [('<s>', 1), (1, 2), (2, 3), (3, 4), (4, 5)]
        >>> list(ngrams([1,2,3,4,5], 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
        [('<s>', 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, '</s>')]


    :param sequence: the source data to be converted into ngrams
    :type sequence: sequence or iter
    :param n: the degree of the ngrams
    :type n: int
    :param pad_left: whether the ngrams should be left-padded
    :type pad_left: bool
    :param pad_right: whether the ngrams should be right-padded
    :type pad_right: bool
    :param left_pad_symbol: the symbol to use for left padding (default is None)
    :type left_pad_symbol: any
    :param right_pad_symbol: the symbol to use for right padding (default is None)
    :type right_pad_symbol: any
    :rtype: sequence or iter
    """
    sequence = pad_sequence(sequence, n, **kwargs)

    # Creates the sliding window, of n no. of items.
    # `iterables` is a tuple of iterables where each iterable is a window of n items.
    iterables = tee(sequence, n)

    for i, sub_iterable in enumerate(iterables):  # For each window,
        for _ in range(i):  # iterate through every order of ngrams
            next(sub_iterable, None)  # generate the ngrams within the window.
    return zip(*iterables)  # Unpack and flattens the iterables.

class Metric():
    """
    Defines a text quality metric.
    """

    def get_name(self):
        return self.name

    @abc.abstractmethod
    def compute_metric(self, texts):
        pass

class Distinct_N(Metric):

    def __init__(self, n):
        self.n = n
        self.name = f'Distinct_{n}'

    def compute_metric(self, texts):
        return self._distinct_ngrams(texts, self.n)

    def _distinct_ngrams(self, texts, n):
        total = 0.0
        for t in texts:
            tokens = nltk.tokenize.word_tokenize(t)
            n_distinct = len(set(ngrams(tokens, n)))
            total += n_distinct/ len(tokens)
        return total / len(texts)

In [35]:
## compute diversity
metrics = {'dist_1': Distinct_N(1), 
            'dist_2': Distinct_N(2), 
            'dist_3': Distinct_N(3),
}
for i, (name, metric) in enumerate(metrics.items()):
    metric_result = metric.compute_metric(generate_codes)
    results[name] = metric_result
    print(metric_result)

NameError: name 'tee' is not defined

In [45]:
def calculate_diversity(text):
    # 将文本转换为字符列表
    chars = list(text)
    
    # 计算字符总数和唯一字符数
    num_chars = len(chars)
    unique_chars = set(chars)
    num_unique_chars = len(unique_chars)
    
    # 计算多样性指数
    diversity = num_unique_chars / num_chars
    
    return diversity


sum_gen=0
list_gen=[]
sum_act=0
list_act=[]

sum_p=0
p=0
list_p=[]

for i in range(50):
    score_gen=calculate_diversity(generate_codes[i])
    list_gen.append(score_gen)
    sum_gen=sum_gen+score_gen
    
    score_act=calculate_diversity(ground_truth_codes[i])
    list_act.append(score_act)
    sum_act=sum_act+score_act
    
    p=score_gen/score_act
    sum_p=sum_p+p
    list_p.append(p)
    
    
sum_gen=sum_gen/50
sum_act=sum_act/50

print('generate_score=',sum_gen)
print('actual_score=',sum_act)
print('p=',sum_p/50)

print('delta_p=',abs(1-sum_p/50))

generate_score= 0.0892122330129225
actual_score= 0.12385099950064055
p= 0.8334312613156932
delta_p= 0.16656873868430677
